In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import datetime
import pandas as pd
import urllib.request

In [81]:
#Getting the page response and parsing the contetnt into html
import requests
page = requests.get("https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c")
soup = bs(page.content,'html.parser')

def into_celcius(temp):
    """A function to convert fahreinheit into celcius"""
    return round((int(temp) - 32) * 5/9, 2)


days = {'Mon':'Monday', 'Tue':'Tuesday','Wed':'Wednesday','Thu':'Thursday','Fri':'Friday','Sat':'Saturday','Sun':'Sunday'}
data = {'day':[], 'date':[],'low_temp_degrees':[],'high_temp_degrees':[],'summary':[]}

date_html = soup.find_all('div',class_ = 'DailyContent--DailyContent--rTQY_') #Extracting the date and day 
for i,val in enumerate(date_html):   
    date = val.h3.span.text
    if i%2==0:
        data['day'].append(days[date.split(' ')[0]])
        data['date'].append(str(2021)+str('-03-')+str(date.split(' ')[1]))

#Extracting the summary of the day.    
div_summary = soup.find_all('div',class_="DaypartDetails--Content--XQooU DaypartDetails--contentGrid--3cYKg")
for val in div_summary:
    summ = val.p.text
    data['summary'].append(summ)    
    
#extracting the max and min temprature.
max_temp_html = soup.find_all('div',class_='DailyContent--ConditionSummary--2vnrT')
for i in range(0,len(max_temp_html)):
    max_temp = max_temp_html[i].div.span.text
    if len(max_temp_html)%2==0:
        if i%2!=0:
            data['low_temp_degrees'].append(into_celcius("".join(list(max_temp_html[i].div.span.text)[0:2])))
        else:    
            data['high_temp_degrees'].append(into_celcius("".join(list(max_temp_html[i].div.span.text)[0:2])))
    else:
        if i == 0:
            data['high_temp_degrees'].append('Not known')
        if i%2==0:
            data['low_temp_degrees'].append(into_celcius("".join(list(max_temp_html[i].div.span.text)[0:2])))
        else:
            data['high_temp_degrees'].append(into_celcius("".join(list(max_temp_html[i].div.span.text)[0:2])))

#Making the dataframe out  of this.
data = pd.DataFrame(data)
data   

,day,date,low_temp_degrees,high_temp_degrees,summary
0,Thursday,2021-03-25,7.22,Not known,Clear skies. Low around 45F. Winds N at 5 to 1...
1,Friday,2021-03-26,5.56,16.67,A mainly sunny sky. High 62F. NNE winds shifti...
2,Saturday,2021-03-27,6.11,17.22,A mainly sunny sky. High 63F. SSE winds shifti...
3,Sunday,2021-03-28,8.33,15.56,"A few passing clouds, otherwise generally sunn..."
4,Monday,2021-03-29,8.89,16.67,A mainly sunny sky. High 62F. Winds WNW at 10 ...
5,Tuesday,2021-03-30,10.00,19.44,Sunny. High 67F. Winds N at 10 to 20 mph.
6,Wednesday,2021-03-31,10.00,21.67,Mainly sunny. High 71F. Winds NNE at 10 to 15 ...
7,Thursday,2021-03-01,8.33,21.11,Mainly sunny. High 69F. NE winds shifting to W...
8,Friday,2021-03-02,7.78,17.22,Partly cloudy. High 63F. Winds W at 10 to 20 mph.
9,Saturday,2021-03-03,7.22,15,Sunshine and clouds mixed. High 59F. Winds W a...
